In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
import scipy as sp
import os
import librosa
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import cv2
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
from keras.models import load_model
from PIL import Image
import csv

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]

for this model, we will train the data using the mel-spectrogram figures. For this, we will first store them as images and then using them, train a 2D CNN which will be used for testing.

In [47]:
data = []
for label in labels : 
    label_data = np.load(label + ".npy")
    for arr in label_data:
        D = np.abs(librosa.stft(arr))**2
        S = librosa.feature.melspectrogram(S=D, sr=16000, n_mels=128, fmax=8000)
        K = librosa.power_to_db(S,ref=np.max)
        temp = [[label, K]]
        data += temp
data = pd.DataFrame(data, columns=["Label", "Spectrogram"])

KeyboardInterrupt: 

In [38]:
dir_path = "training_set"
print(os.listdir())
if(dir_path not in os.listdir()):
    os.mkdir(dir_path)
    for i in labels:
        os.mkdir(os.path.join(dir_path, i))

prev = np.zeros((128,32))
for i in range(data.shape[0]):
    label = data.iloc[i].Label
    spectrogram = data.iloc[i].Spectrogram
    print(prev == spectrogram) 
    prev = spectrogram
    plt.imsave(os.path.join(dir_path, label, str(i) + ".png"), spectrogram)
    

['.ipynb_checkpoints', 'another_model.ipynb', 'checkpoints', 'down.npy', 'filenameset.npy', 'go.npy', 'left.npy', 'link_to_gcp_credits_form.txt', 'model3.ipynb', 'New folder', 'no.npy', 'off.npy', 'on.npy', 'resimages.npy', 'reslabels.npy', 'right.npy', 'sample_submission', 'sample_submission.7z', 'Second.ipynb', 'silence.npy', 'spectrogram_training_set.ipynb', 'stop.npy', 'Tensor Flow Challenge.ipynb', 'test', 'test.7z', 'testingset.npy', 'test_images', 'train', 'train.7z', 'training_again-Copy1.ipynb', 'training_again.ipynb', 'training_data.ipynb', 'training_set', 'unknown', 'unknown.npy', 'Untitled.ipynb', 'up.npy', 'x_train.npy', 'yes.npy', 'y_res.npy', 'y_train.npy', '_init_.ipynb']


NameError: name 'data' is not defined

In [41]:
resimages = [] 
reslabels = []
dir_path = "training_set"
k=0
prev = np.zeros((128,32))
for label in os.listdir(dir_path):
    for file in os.listdir(os.path.join(dir_path, label)):
        img = cv2.imread(os.path.join(dir_path, label, file))
#         print(img.shape)
#         print(np.array_equal(prev, img))
        prev = img
        resimages.append(img)
        reslabels.append(label)
        k+=1

In [5]:
resimages = np.array(resimages)
reslabels = np.array(reslabels)
print(resimages.shape)
print(reslabels.shape)
np.save("resimages", resimages)
np.save("reslabels", reslabels)

NameError: name 'resimages' is not defined

In [6]:
train_images = np.load("resimages.npy")
reslabels = np.load("reslabels.npy")
# print(reslabels)
le = LabelEncoder()
train_labels = le.fit_transform(reslabels)
# print(train_labels)
# print(le.inverse_transform(train_labels))

In [13]:
xtr, xte, ytr, yte = model_selection.train_test_split(train_images, reslabels, test_size=0.2)
def create_model() :
    model = models.Sequential()
    model.add(layers.Conv2D(128 , (3,3), activation='softmax', input_shape=(128, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128 , (3,3), activation='softmax', input_shape=(128, 32, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(12))
    
    return model

In [ ]:
print(reslabels.shape)
print(train_images.shape)
model = create_model()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,  batch_size=100 )

In [ ]:
score = model.evaluate(xtr, ytr, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(xte, yte, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

The maximum accuracy achieved by this model was 85% on the training set. Hence we will use this model on our testing set.

In [ ]:
model.save_weights('./checkpoints/my_checkpoint_3')

In [86]:
model_new = create_model()
model_new.load_weights('./checkpoints/my_checkpoint')

In [16]:
testing_folder = os.path.join("test", "audio")
print(len(os.listdir(testing_folder)))


158538


In [ ]:
destination_folder = os.path.join("test_images")
if(destination_folder not in os.listdir()):
    os.mkdir(destination_folder)
testingset = []
filenameset = []
prev = np.zeros((128,32, 3))
prev_file = ""
for audio_file in os.listdir(testing_folder):
    file_name = audio_file[0:-4]
    audio_file_path = os.path.join(testing_folder, audio_file)
    audio_file_arr, sr = librosa.load(audio_file_path, sr=16000)
    conv_numpy = librosa.util.fix_length(audio_file_arr, 16000)
    D = np.abs(librosa.stft(audio_file_arr))**2
    S = librosa.feature.melspectrogram(S=D, sr=16000, n_mels=128, fmax=8000)
    K = librosa.power_to_db(S,ref=np.max)
    plt.imsave(os.path.join(destination_folder, file_name+".png"), K)
    img = cv2.imread(os.path.join(destination_folder, file_name + ".png"))
    prev = img
    prev_file = file_name
    testingset.append(img)
    filenameset.append(audio_file + ".wav")

In [55]:
print(len(testingset))
testing_set = np.array(testingset)
filename_set  = np.array(filenameset)
print(testing_set.shape)
print(filename_set.shape)

158538
(158538, 128, 32, 3)
(158538,)


In [56]:
np.save("testing_set", testing_set)
np.save("filename_set", filename_set)

In [18]:

testing_set = np.load("testing_set.npy")
filename_set = np.load("filename_set.npy")
filename = "submission.csv"

In [ ]:
y_res = model_new.predict_classes(testingset)

In [87]:
y_res_2 = np.argmax(model_new.predict(testing_set), axis=-1)
# print(le.inverse_transform(y_res))
# print(le.inverse_transform(y_res_2))

In [88]:
print(y_res_2)

[3 9 5 ... 5 7 2]


In [89]:
result = np.array(y_res_2)

np.save("result", result)

In [90]:
res = np.load("result.npy")
res_label = le.inverse_transform(res)

In [91]:
file_data = []

for i in range(len(res)):
    file_data.append([filename_set[i][0:-4], res_label[i]])

In [92]:
for i in range(100):
    print(file_data[i])

['clip_000044442.wav', 'no']
['clip_0000adecb.wav', 'unknown']
['clip_0000d4322.wav', 'on']
['clip_0000fb6fe.wav', 'unknown']
['clip_0001d1559.wav', 'unknown']
['clip_0002256ed.wav', 'unknown']
['clip_0002a4a1f.wav', 'on']
['clip_0002d9b83.wav', 'unknown']
['clip_000373a5b.wav', 'go']
['clip_0003c7122.wav', 'right']
['clip_0003e6aee.wav', 'off']
['clip_00049951d.wav', 'unknown']
['clip_0004c6707.wav', 'off']
['clip_0004f8b63.wav', 'left']
['clip_00068e281.wav', 'no']
['clip_00069e9cb.wav', 'stop']
['clip_0006f7b8b.wav', 'left']
['clip_0007ec7e6.wav', 'down']
['clip_00082a7d6.wav', 'unknown']
['clip_00094eb22.wav', 'on']
['clip_000962cbb.wav', 'unknown']
['clip_000a96d0a.wav', 'unknown']
['clip_000b01093.wav', 'unknown']
['clip_000b378f1.wav', 'unknown']
['clip_000b9514b.wav', 'unknown']
['clip_000c07b07.wav', 'unknown']
['clip_000c2c07b.wav', 'unknown']
['clip_000c41da7.wav', 'on']
['clip_000dcdd2c.wav', 'silence']
['clip_000ed5715.wav', 'down']
['clip_000f49605.wav', 'unknown']
['clip

In [93]:
print(len(file_data))

158538


In [94]:
data = pd.DataFrame(file_data, columns = ["Fname", "label"])

In [95]:
data.to_csv('Submission_2.csv', index = None)